In [1]:
# Install required Python libraries
try:
    import subprocess
    subprocess.check_call(["pip", "install", "torch", "onnx", "numpy", "hummingbird-ml"])
except:
    print("Ensure all dependencies are installed.")

from sklearn.linear_model import LogisticRegression
from hummingbird.ml import convert
import numpy as np
import json
import torch

In [ ]:
!wget https://github.com/chris-chris/mina-zkml/releases/latest/download/mina-zkml-cli

In [3]:
# here we create and (potentially train a model)
# make sure you have the dependencies required here already installed
import numpy as np
from sklearn.linear_model import LogisticRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LogisticRegression().fit(X, y)
reg.score(X, y)
model = convert(reg, "torch", X[:1]).model
onnx_path = "logistic_regression.onnx"

# Export ONNX model

# Input to the model
shape = X.shape[1:]
dummy_input = torch.rand(1, *shape, requires_grad=True)
torch_out = circuit(dummy_input)

torch.onnx.export(
    model, dummy_input, onnx_path,
    input_names=["input"], output_names=["output"],
    opset_version=13
)

torch.onnx.export(circuit,               # model being run
                  # model input (or a tuple for multiple inputs)
                  dummy_input,
                  # where to save the model (can be a file or file-like object)
                  onnx_path,
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

print(f"ONNX model exported to {onnx_path}")


ONNX model exported to logistic_regression.onnx


In [4]:
# Prepare input data for proof generation
data_path = "input.json"
sample_input = X[:1]  # Use one sample from the dataset
input_data = sample_input.flatten().tolist()  # Flatten for JSON serialization

# Save the input data to a JSON file
data = [input_data]  # Wrap in outer array [[]]
with open(data_path, "w") as f:
    json.dump(data, f, indent=4)
print(f"Input data saved to {data_path}")


Input data saved to input.json


In [6]:
import subprocess

# Paths for CLI commands
proof_path = "proof.json"

# Command for proof generation
# Make sure to generate the binary first: `cargo build --release`
cmd = [
    "/content/mina-zkml-cli", "proof",
    "-m", onnx_path,
    "-i", data_path,
    "-o", proof_path,
    "--input-visibility", "public",
    "--output-visibility", "public"
]

# Run the CLI command
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
if result.returncode == 0:
    print(f"Proof successfully generated at {proof_path}")
else:
    print(f"Error generating proof: {result.stderr}")


Node: Node { id: 0, name: "input", inputs: [], op: TypedSource { fact: 1,2,F32 }, outputs: [1,2,F32 >2/0] }
Found Input operation
Node: Node { id: 1, name: "_operators.0.coefficients.0", inputs: [], op: Const(2,4,F32 -0.2899942, -0.41289154, 0.4130071, 0.28987864, -0.6468678, 0.06529999, -0.06524275, 0.6468106), outputs: [2,4,F32 -0.2899942, -0.41289154, 0.4130071, 0.28987864, -0.6468678, 0.06529999, -0.06524275, 0.6468106 >2/1] }
Found Const operation
Node: Node { id: 2, name: "/_operators.0/Gemm.ab", inputs: [0/0>, 1/0>], op: EinSum mk,kn->mn (F32), outputs: [1,4,F32 >4/0] }
Found matrix operation: EinSum
Node: Node { id: 3, name: "/_operators.0/Gemm.c_add_axis_1", inputs: [], op: Const(1,4,F32 1.6625432, 0.5548876, -0.42277858, -1.7946522), outputs: [1,4,F32 1.6625432, 0.5548876, -0.42277858, -1.7946522 >4/1] }
Found Const operation
Node: Node { id: 4, name: "/_operators.0/Gemm", inputs: [2/0>, 3/0>], op: TypedBinOp(Add, None), outputs: [1,4,F32 >5/0 >9/0] }
Found Add operation: Add

In [7]:
# Extract the "output" field from proof.json
output_path = "output.json"
try:
    with open(proof_path, "r") as proof_file:
        proof_data = json.load(proof_file)
    if "output" in proof_data:
        output_data = proof_data["output"]
        with open(output_path, "w") as output_file:
            json.dump(output_data, output_file, indent=4)
        print(f"Output data successfully saved to {output_path}")
    else:
        print("No 'output' field found in proof.json")
except Exception as e:
    print(f"An error occurred: {e}")


Output data successfully saved to output.json


In [8]:
# Create a public output file from the proof
try:
    # Load proof.json
    with open(proof_path, "r") as proof_file:
        proof_data = json.load(proof_file)

    # Extract the "output" field
    if "output" in proof_data:
        output_data = proof_data["output"]
        
        # Save the output data to output.json
        with open(output_path, "w") as output_file:
            json.dump(output_data, output_file, indent=4)
        
        print(f"Output data successfully saved to {output_path}")
    else:
        print("No 'output' field found in proof.json")
except Exception as e:
    print(f"An error occurred: {e}")

Output data successfully saved to output.json


In [9]:
# Command for proof verification
cmd = [
    "/content/mina-zkml-cli"", "verify",
    "-m", onnx_path,
    "-i", data_path,
    "-p", proof_path,
    "-o", output_path,
    "--input-visibility", "public",
    "--output-visibility", "public"
]

# Run the CLI command
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
if result.returncode == 0:
    print(f"Proof successfully verified at {proof_path}")
else:
    print(f"Error verifying proof: {result.stderr}")


Node: Node { id: 0, name: "input", inputs: [], op: TypedSource { fact: 1,2,F32 }, outputs: [1,2,F32 >2/0] }
Found Input operation
Node: Node { id: 1, name: "_operators.0.coefficients.0", inputs: [], op: Const(2,4,F32 -0.2899942, -0.41289154, 0.4130071, 0.28987864, -0.6468678, 0.06529999, -0.06524275, 0.6468106), outputs: [2,4,F32 -0.2899942, -0.41289154, 0.4130071, 0.28987864, -0.6468678, 0.06529999, -0.06524275, 0.6468106 >2/1] }
Found Const operation
Node: Node { id: 2, name: "/_operators.0/Gemm.ab", inputs: [0/0>, 1/0>], op: EinSum mk,kn->mn (F32), outputs: [1,4,F32 >4/0] }
Found matrix operation: EinSum
Node: Node { id: 3, name: "/_operators.0/Gemm.c_add_axis_1", inputs: [], op: Const(1,4,F32 1.6625432, 0.5548876, -0.42277858, -1.7946522), outputs: [1,4,F32 1.6625432, 0.5548876, -0.42277858, -1.7946522 >4/1] }
Found Const operation
Node: Node { id: 4, name: "/_operators.0/Gemm", inputs: [2/0>, 3/0>], op: TypedBinOp(Add, None), outputs: [1,4,F32 >5/0 >9/0] }
Found Add operation: Add

## Deployment

For deploying this model, in a Mina network, we need to follow the steps below:

1. Clone `https://github.com/chris-chris/mina-zkml-verifier.git` repository.
2. Move to the `mina-zkml-verifier` directory.
3. Run `npm install` to install the dependencies.
4. Follow the network setup instructions in the `README.md` file in the mina-zkml-verifier repository.
5. Update the `modelConfig.json` file with the correct values, and the model in onnx format.
6. Run `zk deploy <NETWORK_NAME>`, the <NETWORK_NAME> is alias for the account you created in step 4.